# Using CLMBR to generate features and training models on those features

We can use a trained CLMBR model to generate features and then use those features in a logistic regression model.

In [1]:
import shutil
import os

TARGET_DIR = 'trash/tutorial_7'

if os.path.exists(TARGET_DIR):
    shutil.rmtree(TARGET_DIR)

os.mkdir(TARGET_DIR)

In [2]:
import femr.models.transformer
import pyarrow.csv
import datasets
import pickle

# First, we compute our features

# Load some labels
labels = pyarrow.csv.read_csv('input/labels.csv').to_pylist()

# Load our data
dataset = datasets.Dataset.from_parquet("input/meds/data/*")

# We need an ontology for MOTOR
with open('input/meds/ontology.pkl', 'rb') as f:
    ontology = pickle.load(f)

features = femr.models.transformer.compute_features(dataset,'input/motor_model', labels, num_proc=4, tokens_per_batch=128, ontology=ontology)

# We have our features
for k, v in features.items():
    print(k, v.shape)

/share/pi/nigam/projects/zphuo/repos/transformers/src/transformers/utils/hub.py:124: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


Map (num_proc=4):   0%|          | 0/200 [00:00<?, ? examples/s]

Some weights of the model checkpoint at input/motor_model were not used when initializing FEMRModel: ['task_model.final_layer.bias', 'task_model.final_layer.weight', 'task_model.task_embedding.weight', 'task_model.task_embedding_bias.weight', 'transformer.in_norm.scale', 'transformer.layers.0.norm.scale', 'transformer.layers.1.norm.scale', 'transformer.out_norm.scale']
- This IS expected if you are initializing FEMRModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing FEMRModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of FEMRModel were not initialized from the model checkpoint at input/motor_model and are newly initialized: ['transformer.in_norm.weight', 'transformer.layers.0.n

<class 'femr.models.transformer.FEMRModel'> model type


Map (num_proc=4):   0%|          | 0/200 [00:00<?, ? examples/s]

Creating batches 4


Generating train split: 0 examples [00:00, ? examples/s]

{'num_patients': tensor([64]), 'num_indices': tensor([64]), 'patient_ids': tensor([[ 88,  88, 122, 122,  70,  70, 174, 174, 194, 194,  87,  87,  86,  86,
          92,  92,  73,  73,  27,  27, 112, 112, 117, 117, 135, 135,  54,  54,
          62,  62, 141, 141, 177, 177, 107, 107,  50,  50,  99,  99, 189, 189,
          36,  36, 173, 173, 191, 191,  35,  35,  89,  89, 181, 181,  77,  77,
         197, 197,  93,  93,  28,  28,  67,  67,  48,  48, 105, 105,  29,  29,
          57,  57,  69,  69, 120, 120, 198, 198,  72,  72,  12,  12,  43,  43,
         106, 106,  83,  83,  49,  49, 140, 140, 190, 190, 167, 167,  95,  95,
          45,  45,  74,  74,  40,  40,  26,  26,   3,   3,   5,   5,  58,  58,
          20,  20, 130, 130,  34,  34, 172, 172,  19,  19, 123, 123, 121, 121,
         182, 182]]), 'offsets': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0

# Joining features and labels

Given a feature set, it's important to be able to join a set of labels to those features.

This can be done with femr.featurizers.join_labels

In [ ]:
import femr.featurizers

features_and_labels = femr.featurizers.join_labels(features, labels)

for k, v in features_and_labels.items():
    print(k, v.shape)

boolean_values (200,)
patient_ids (200,)
times (200,)
features (200, 64)


# Data Splitting

When using a pretrained CLMBR model, we have to be very careful to use the splits used for the original model

In [ ]:
import femr.splits
import numpy as np

# We split into a global training and test set
split = femr.splits.PatientSplit.load_from_csv('input/motor_model/main_split.csv')

train_mask = np.isin(features_and_labels['patient_ids'], split.train_patient_ids)
test_mask = np.isin(features_and_labels['patient_ids'], split.test_patient_ids)

percent_train = .70
X_train, y_train = (
    features_and_labels['features'][train_mask],
    features_and_labels['boolean_values'][train_mask],
)
X_test, y_test = (
    features_and_labels['features'][test_mask],
    features_and_labels['boolean_values'][test_mask],
)

# Building Models

The generated features can then be used to build your standard models. In this case we construct both logistic regression and XGBoost models and evaluate them.

Performance is perfect since our task (predicting gender) is 100% determined by the features

In [ ]:
import xgboost as xgb
import sklearn.linear_model
import sklearn.metrics
import sklearn.preprocessing

def run_analysis(title: str, y_train, y_train_proba, y_test, y_test_proba):
    print(f"---- {title} ----")
    print("Train:")
    print_metrics(y_train, y_train_proba)
    print("Test:")
    print_metrics(y_test, y_test_proba)

def print_metrics(y_true, y_proba):
    y_pred = y_proba > 0.5
    auroc = sklearn.metrics.roc_auc_score(y_true, y_proba)
    aps = sklearn.metrics.average_precision_score(y_true, y_proba)
    accuracy = sklearn.metrics.accuracy_score(y_true, y_pred)
    f1 = sklearn.metrics.f1_score(y_true, y_pred)
    print("\tAUROC:", auroc)
    print("\tAPS:", aps)
    print("\tAccuracy:", accuracy)
    print("\tF1 Score:", f1)


model = sklearn.linear_model.LogisticRegressionCV(penalty="l2", solver="liblinear").fit(X_train, y_train)
y_train_proba = model.predict_proba(X_train)[::, 1]
y_test_proba = model.predict_proba(X_test)[::, 1]
run_analysis("Logistic Regression", y_train, y_train_proba, y_test, y_test_proba)

---- Logistic Regression ----
Train:
	AUROC: 1.0
	APS: 1.0
	Accuracy: 1.0
	F1 Score: 1.0
Test:
	AUROC: 1.0
	APS: 1.0
	Accuracy: 1.0
	F1 Score: 1.0
